O CEMADEN possui uma API na qual conseguimos acessar os dados dos pluviômetros em tempo real para cada uma das estações. Nessa API não vem a latitude e longitude das estações, mas que podemos obter manualmente do mapa interativo no site. Entretanto o código das estações é diferente nessas duas fontes e o tratamento para identificarmos a latlon das estações está nesse notebook. Consideramos como id_estacao o que é fornecido via API.

Fontes:
    - API: http://sjc.salvar.cemaden.gov.br/resources/graficos/interativo/getJson2.php?uf=RJ
    - arquivo estacoes_cemaden_site.csv: foi baixado em http://www2.cemaden.gov.br/mapainterativo/ > Download de dados > estações pluviométricas

In [6]:
import pandas as pd
import unidecode

df = pd.read_csv("estacoes_cemaden_site.csv", sep=';')
df.head()

,municipio,codEstacao,uf,nomeEstacao,latitude,longitude,datahora,valorMedida
0,RIO DE JANEIRO,330455701A,RJ,Glória,"-22,915","-43,176",2023-07-01 00:20:00.0,"0,00"
1,RIO DE JANEIRO,330455701A,RJ,Glória,"-22,915","-43,176",2023-07-01 00:50:00.0,"0,00"
2,RIO DE JANEIRO,330455701A,RJ,Glória,"-22,915","-43,176",2023-07-01 01:50:00.0,"0,00"
3,RIO DE JANEIRO,330455701A,RJ,Glória,"-22,915","-43,176",2023-07-01 02:50:00.0,"0,00"
4,RIO DE JANEIRO,330455701A,RJ,Glória,"-22,915","-43,176",2023-07-01 03:50:00.0,"0,00"


In [7]:
df = df[df['municipio'] == 'RIO DE JANEIRO'].copy()
df = df.drop(columns=['valorMedida', 'municipio', 'uf', 'datahora']).drop_duplicates()
df.shape

(26, 4)

In [8]:
df[['latitude', 'longitude']] = df[['latitude', 'longitude']].apply(lambda x: x.replace({",": '.'}, regex=True))
df.head()

,codEstacao,nomeEstacao,latitude,longitude
0,330455701A,Glória,-22.915,-43.176
776,330455704A,Praça Seca,-22.896,-43.352
1397,330455705A,Tanque Jacarepagua,-22.918,-43.361
2177,330455706A,Penha,-22.839,-43.279
2942,330455709A,Est. Pedra Bonita,-22.977861,-43.277444


In [9]:
df.rename(columns={'codEstacao':'id_estacao', "nomeEstacao":"estacao"}, inplace=True)
df.estacao.unique()

array(['Glória', 'Praça Seca', 'Tanque Jacarepagua', 'Penha',
       'Est. Pedra Bonita', 'Jardim Maravilha', 'Santa Cruz',
       'Realengo Batan', 'Padre Miguel', 'Salgueiro', 'Andaraí',
       'CIEP Samuel Wainer', 'Vargem Pequena', 'Jacarepaguá',
       'CIEP Dr. João Ramos de Souza', 'São Conrado', 'Catete', 'Pavuna',
       'Vigário Geral', 'Vicente de Carvalho', 'Alto da Boa Vista',
       'Tijuca', 'Usina', 'Higienópolis', 'Pilares', 'Ilha de Paquetá'],
      dtype=object)

In [15]:
df['estacao'] = df['estacao'].str.normalize('NFKD').str.encode('ascii',errors='ignore').str.decode('utf-8').str.capitalize()

Pegar dados direto da API

In [10]:
url = "http://sjc.salvar.cemaden.gov.br/resources/graficos/interativo/getJson2.php?uf=RJ"
dados = pd.read_json(url)

rename_cols = {
    "idestacao": "id_estacao",
     "nomeestacao":"estacao",
}

drop_cols = [
        "ultimovalor",
        "datahoraUltimovalor",
        "acc1hr",
        "acc3hr",
        "acc6hr",
        "acc12hr",
        "acc24hr",
        "acc48hr",
        "acc72hr",
        "acc96hr",
        "uf",
        "codibge",
        "cidade",
        "tipoestacao",
        "status",
        ]
dados = (
    dados[(dados["codibge"] == 3304557) & (dados["tipoestacao"] == 1)]
    .drop(drop_cols, axis=1)
    .rename(rename_cols, axis=1)
)
dados.head()

,id_estacao,estacao
14,3044,Penha
16,7609,Vicente de Carvalho
21,7614,Pilares
26,7611,Tijuca
41,7592,Santa Cruz


In [11]:
dados['estacao'] = dados['estacao'].str.normalize('NFKD').str.encode('ascii',errors='ignore').str.decode('utf-8').str.capitalize()

Junta dados da API com o arquivo do site para conseguir latlon das estações

In [16]:
dfr = df.drop(columns='id_estacao').merge(dados, on='estacao', how='outer')
dfr.head()

,estacao,latitude,longitude,id_estacao
0,Gloria,-22.915,-43.176,3114
1,Praca seca,-22.896,-43.352,3045
2,Tanque jacarepagua,-22.918,-43.361,3043
3,Penha,-22.839,-43.279,3044
4,Est. pedra bonita,-22.977861,-43.277444,3215


In [17]:
dfr.tail()

,estacao,latitude,longitude,id_estacao
23,Higienopolis,-22.869,-43.265,7613
24,Pilares,-22.875,-43.299,7614
25,Ilha de paqueta,-22.7613,-43.1084,7615
26,Defesa civil santa cruz,NaN,NaN,7607
27,Abolicao,NaN,NaN,3042


In [18]:
dfr.loc[dfr.estacao=='Defesa civil santa cruz', ['latitude', 'longitude']] = -22.89444, -43.67611
# https://comiteguandu.org.br/conteudo/estacao-rede-basica-2021.pdf

In [19]:
dfr.tail()

,estacao,latitude,longitude,id_estacao
23,Higienopolis,-22.869,-43.265,7613
24,Pilares,-22.875,-43.299,7614
25,Ilha de paqueta,-22.7613,-43.1084,7615
26,Defesa civil santa cruz,-22.89444,-43.67611,7607
27,Abolicao,NaN,NaN,3042


In [20]:
dfr[[ 'id_estacao', 'estacao', 'latitude', 'longitude']].sort_values("id_estacao").to_csv("estacoes_cemaden.csv", index=False)